In [1]:
from sqlalchemy import create_engine, MetaData

import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats

from shapely.geometry import Polygon
from shapely.ops import transform
import pyproj

#from geoalchemy2 import Geometry  # <= not used but must be imported

In [4]:
# to bring all functions that we have develop for marxan.
%run marxan_utils.ipynb

Files that we need to create by default: 
* input.dat
* input
    * puvsp.dat (PlanningUnitVSConservationFeature)
    * spec.dat (conservationFeature)
    * pu.dat (planningUnits)
    --- only if certain conditions in input.dat are met
    * bound.dat (boundaryLength)
    * .dat (blockDefinition)
* output


```sql
--input.dat
select number_of_runs from scenario where id = 
```

If conditions are not met:

In [8]:
inputDatFile?

Init signature:
inputDatFile(
    *,
    VERSION: str = '0.1',
    BLM: float = 0.0,
    PROP: float = 0.0,
    RANDSEED: int = -1,
    NUMREPS: int = 1,
    BESTSCORE: int = 0,
    NUMITNS: int = 0,
    STARTTEMP: int = 1,
    COOLFAC: int = 0,
    NUMTEMP: int = 1,
    COSTTHRESH: float = 0,
    THRESHPEN1: float = 0,
    THRESHPEN2: float = 0,
    INPUTDIR: str = 'input',
    SPECNAME: str = 'spec.dat',
    PUNAME: str = 'pu.dat',
    PUVSPRNAME: str = 'puvspr2.dat',
    BOUNDNAME: str = 'bound.dat',
    BLOCKDEFNAME: str = 'blockdef.dat',
    VERBOSITY: int = 1,
    MISSLEVEL: float = 1,
    OUTPUTDIR: str = 'output',
    SCENNAME: str = 'Default_name',
    SAVERUN: int = 3,
    SAVEBEST: int = 3,
    SAVESUMMARY: int = 3,
    SAVESCEN: int = 3,
    SAVETARGMET: int = 3,
    SAVESUMSOLN: int = 3,
    SAVELOG: int = 1,
    SAVESNAPSTEPS: int = 0,
    SAVESNAPCHANGES: int = 0,
    SAVESNAPFREQUENCY: int = 0,
    SAVESOLUTIONSMATRIX: int = 3,
    RUNMODE: int = 1,
    ITIMPTYPE: int =